In [2]:
import pandas as pd
from natsort import natsort_keygen
import os
from tqdm import tqdm

In [31]:
chromatin_profiles_dir = "/data2/sei-framework/output/Korean_ASD_WGS/hg38/controls/chromatin_profile_diffs.tsv"
outdir = "/data2/sei-framework/output/Korean_ASD_WGS/hg38/controls"

In [32]:
chromatin_profiles = pd.read_csv(chromatin_profiles_dir, sep="\t", compression="gzip")
max_profiles = pd.DataFrame(columns=["chrom", "pos", "id", "ref", "alt", "max_chromatin_profile", "seqclass_max_absdiff", "chromatin_profiles_score"])

In [33]:
for i in tqdm(chromatin_profiles.index):
    var = chromatin_profiles.loc[i]
    profiles = dict(map(reversed, dict(var[9:].abs()).items()))
    max_score = max(profiles.keys())
    max_class = profiles[max_score]
    seq_score = var[max_class]

    max_profiles = max_profiles.append({"chrom":var["chrom"], "pos":var["pos"], "id":var["id"], "ref":var["ref"],
                                        "alt":var["alt"], "max_chromatin_profile":max_class, "seqclass_max_absdiff":max_score, "chromatin_profile_score":seq_score}, ignore_index=True)

max_profiles.sort_values(["chrom","pos"], key=natsort_keygen(), inplace=True)
max_profiles.to_csv(os.path.join(outdir, "max_chromatin_profile_scores.tsv"), sep="\t", index=False)

max_profiles["SampleID"] = max_profiles.id.str.split(";").str[1].str.split("=").str[1]
max_profiles_cnt = pd.pivot_table(data=max_profiles, index='max_chromatin_profile', columns='SampleID', values='id', aggfunc='count')
max_profiles_cnt.fillna(0, inplace=True)
max_profiles_cnt = max_profiles_cnt.applymap(int)
max_profiles_cnt.insert(0, "max_chromatin_profile", max_profiles_cnt.index)
max_profiles_cnt.reset_index(drop=True, inplace=True)
max_profiles_cnt.sort_values("max_chromatin_profile", key=natsort_keygen(), inplace=True)

max_profiles_cnt.to_csv(os.path.join(outdir, "max_chromatin_profile_counts_per_sample.tsv"), sep="\t", index=False)

print("Done!")

100%|██████████| 13503/13503 [17:33<00:00, 12.82it/s]


Done!


In [35]:
max_profiles_cnt

SampleID,max_chromatin_profile,10234.s,10244.s,10524.s,12694.s,12744.s,12774.s,12794.s,12804.s,12814.s,...,9645.s,9734.s,9735.s,9814.s,9864.s,9865.s,9866.s,9884.s,9904.s,9905.s
17,4star | H3K4me1 | Roadmap,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
18,4star | H3K9me3 | Roadmap,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,9.4_Neuron_Prefrontal | ATAC-seq | ID:78242,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22RV1_Epithelium_Prostate | FOXA1 | ID:81170,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22RV1_Epithelium_Prostate | FOXA1 | ID:87801,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2757,neural_crest_line | H3K27ac | ID:83396,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2758,occipital_lobe | DNase | ID:64687,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2759,osteosarcoma | H3K4me1 | ID:82770,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2760,osteosarcoma | H3K4me1 | ID:87302,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [3]:
case_cp = pd.read_csv("/data2/sei-framework/output/Korean_ASD_WGS/hg38/cases/max_chromatin_profile_counts_per_sample.tsv", sep="\t")
ctrl_cp = pd.read_csv("/data2/sei-framework/output/Korean_ASD_WGS/hg38/controls/max_chromatin_profile_counts_per_sample.tsv", sep="\t")

In [5]:
all_x = pd.merge(case_cp, ctrl_cp, on="max_chromatin_profile", how="outer")
all_x.fillna(0, inplace=True)
all_x.iloc[:, 1:] = all_x.iloc[:, 1:].applymap(int)
#all_x.to_csv("/data2/sei-framework/result/Korean_ASD_WGS/hg38/data/NMF/ASD_cases+controls/Korean_ASD_WGS_872samples_DNM_max_chromtin_profiles_raw_freq.tsv", sep="\t", index=False)

In [6]:
adj_file = pd.read_csv("/home/sonic/ASD_samples/Korean/adjustFile_patAgeOnly_Korean_ASD_WGS_v3_final_autosome_DNV_874_samples.20220816.txt", sep="\t")
adj_file

,SampleID,SAMPLE,FatherAgeBirth,MotherAgeBirth,N_dnv,N_dnv_adjRaw,N_dnv_adj,AdjustFactor
0,IBS-ASD-10003-blood-wgs-ILLUMINA,10003.p,373.0,369.0,64,9.881334,69.199484,1.081242
1,IBS-ASD-10233-blood-wgs-ILLUMINA,10233.p,457.0,433.0,54,-12.306247,47.011903,0.870591
2,IBS-ASD-10234-blood-wgs-ILLUMINA,10234.s,457.0,433.0,72,5.693753,65.011903,0.902943
3,IBS-ASD-10243-blood-wgs-ILLUMINA,10243.p,378.0,285.0,51,-3.844117,55.474033,1.087726
4,IBS-ASD-10244-blood-wgs-ILLUMINA,10244.s,397.0,304.0,51,-6.600832,52.717318,1.033673
...,...,...,...,...,...,...,...,...
869,IBS-ASD-7463-blood-wgs-ILLUMINA,7463.p,NaN,NaN,43,NaN,NaN,1.000000
870,IBS-ASD-7553-blood-wgs-ILLUMINA,7553.p,NaN,NaN,78,NaN,NaN,1.000000
871,IBS-ASD-7583-blood-wgs-ILLUMINA,7583.p,NaN,NaN,64,NaN,NaN,1.000000
872,IBS-ASD-7613-blood-wgs-ILLUMINA,7613.p,NaN,NaN,68,NaN,NaN,1.000000


In [9]:
adj_factor = adj_file.loc[:, ["SAMPLE","AdjustFactor"]]
adj_factor

,SAMPLE,AdjustFactor
0,10003.p,1.081242
1,10233.p,0.870591
2,10234.s,0.902943
3,10243.p,1.087726
4,10244.s,1.033673
...,...,...
869,7463.p,1.000000
870,7553.p,1.000000
871,7583.p,1.000000
872,7613.p,1.000000


In [22]:
samples = all_x.columns[1:]
all_adj_x = all_x.copy()

for s in samples:
    adj_f = float(adj_factor.loc[adj_factor.SAMPLE==s, "AdjustFactor"])
    all_adj_x[s] = all_x[s] * adj_f

all_adj_x.to_csv("/data2/sei-framework/result/Korean_ASD_WGS/hg38/data/NMF/ASD_cases+controls/Korean_ASD_WGS_872samples_DNM_max_chromtin_profiles_adj_freq.tsv", sep="\t", index=False)

In [20]:
adj_f = adj_factor.loc[adj_factor.SAMPLE=="10003.p", "AdjustFactor"]
print(adj_f)
#all_x["10003.p"]

0    1.081242
Name: AdjustFactor, dtype: float64
